In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

In [2]:
wine_df = pd.read_csv('winequality-red.csv')

# **Критерий КОЛМОГОРОВА-СМИРНОВА**

**запишем функцию для вычисления**

In [3]:
def kolmogorov(data, loc, scale):
    D_plus = np.zeros(data.shape[0])
    D_minus = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        D_plus[i] = ((i + 1) / data.shape[0]) - st.norm.cdf(data[i], loc=loc, scale=scale)
        D_minus[i] = st.norm.cdf(data[i], loc=loc, scale=scale) - ((i) / data.shape[0])
    D = np.max([np.max(D_plus), np.max(D_minus)])
    return (6 * data.shape[0] * D + 1) / (6 * np.sqrt(data.shape[0]))

**вычислим критерий соответствия данных wine_df['pH'] с нормальным распределением**

**Для начала определим матожидание. Применим 95% доверительный интервал. Берем Z статистику, поскольку у нас много значений в выборке**

In [4]:
Cl = wine_df['pH'].mean() - 1.96 * np.std(wine_df['pH'], ddof=1) / np.sqrt(wine_df['pH'].shape[0])
Cr = wine_df['pH'].mean() + 1.96 * np.std(wine_df['pH'], ddof=1) / np.sqrt(wine_df['pH'].shape[0])
print(Cl, Cr)
print(f"прикидочное среднее: {np.mean(np.array([Cl, Cr]))}")

3.303545893815603 3.318680497679083
прикидочное среднее: 3.311113195747343


**черт с ним, будем брать среднее и дисперсию выборкию Посчитаем Колмогорова**

In [5]:
kolmogorov(wine_df['pH'], loc=wine_df['pH'].mean(), scale=np.std(wine_df['pH'], ddof=1))

38.86358450482914

**Че то много. Протестируем Колмогорова на заведомо нормально распределенной выборке**

In [6]:
test_data = st.norm.rvs(loc=5, scale=7, size=2000)
kolmogorov(test_data, loc=test_data.mean(), scale=np.std(test_data, ddof=1))

44.18626306387382

**??????????????????????????????????????????????????????????????????????????????????????????????????????**

**попробуем воспользоваться scipy**

In [15]:
st.kstest(wine_df['pH'], st.norm.cdf, args=(wine_df['pH'].mean(), np.std(wine_df['pH'], ddof=1)))

KstestResult(statistic=0.040368454197074355, pvalue=0.010604884802440944)

In [13]:
st.kstest(test_data, st.norm.cdf, args=(5, 6.9))

KstestResult(statistic=0.02344463304844213, pvalue=0.21818490719902595)

# **Критерий Омега-квадрат**

**напишем функцию вычисления статистики омега квадрат**

In [9]:
def omega_square(data, loc, scale):
    return (1 / 12 / data.shape[0]) + np.sum([(st.norm.cdf(data[i], loc=loc, scale=scale) - (2 * i) / (2 * data.shape[0])) ** 2 for i in range(data.shape[0])])

**вычислим статистику омега квадрат для столбца density**

In [10]:
omega_square(wine_df['density'], np.mean(wine_df['density']), np.std(wine_df['density'], ddof=1))

358.9006714768003

In [11]:
test_data = st.norm.rvs(loc=5, scale=7, size=2000)
omega_square(test_data, test_data.mean(), np.std(test_data, ddof=1))

340.448333825846

**????????????????????????????????**